# Policy with data connections

This notebook will show how to create a policy that performs API calls to external systems

In [ ]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

In [ ]:
from lineart_sdk import Lineart

BASE_URL = "http://localhost:6001"

In [ ]:
with Lineart(server_url=BASE_URL) as client:
    policy = client.policies.create(
        name="DataConnectionsDemo",
        description="Demo policy with data connections",
    )
    policy_id = policy.policy_id

policy

In [ ]:
from lineart_sdk.models import (
    CachingOptions,
    EnvVarConfig,
    HTTPSource,
    ResponseType,
    RetryPolicy,
    RunTimeParam,
)

api = "my-api"
version = "1.0"
MY_API_REF = f"{api}-v{version}"

source = HTTPSource(
    url="http://testdata:5000/",
    method="GET",
    headers={"Content-Type": "application/json"},
    params={"full": EnvVarConfig(env="FULL")},
    body={"tax_id": RunTimeParam(param="tax_id")},
    retry=RetryPolicy(
        max_retries=3,
        backoff_factor=1,
    ),
    timeout=5,
    response_type=ResponseType.JSON.value,
)

caching = CachingOptions(enabled=True, ttl=3600)


with Lineart(server_url=BASE_URL) as client:
    data_source = client.data_sources.create(
        name=MY_API_REF,
        description="Test data source",
        source=source,
        caching=caching,
        metadata={"api": api, "version": version},
    )
data_source

In [ ]:
from vulkan.spec.dependency import INPUT_NODE, Dependency
from vulkan.spec.nodes import BranchNode, ConnectionNode, DataInputNode, TerminateNode
from vulkan.spec.policy import PolicyDefinition

In [ ]:
def _format_name(name: str) -> str:
    # Replace every character not in regex "^[A-Za-z0-9_]+$" with _
    return "".join(c if c.isalnum() or c == "_" else "_" for c in name)


# Format the data source name into a valid node name
MY_API_DATA_INPUT = _format_name(MY_API_REF)

In [ ]:
api = DataInputNode(
    name=MY_API_DATA_INPUT,
    data_source=MY_API_REF,
    parameters={"tax_id": "'{{inputs.tax_id}}'"},
    dependencies={"inputs": Dependency(INPUT_NODE)},
)


def make_decision(context, scores, **kwargs):
    context.log.info(f"Scores: {scores}")
    if scores["scr"] > 600:
        return "approved"
    if scores["serasa"] > 800:
        return "analysis"
    return "denied"


decision = BranchNode(
    name="decision",
    func=make_decision,
    choices=["approved", "analysis", "denied"],
    dependencies={"scores": Dependency(api.name)},
)

output_data = {"scores": f"{{{{{api.name}}}}}"}

approved = TerminateNode(
    name="approved",
    return_status="approved",
    output_data=output_data,
    dependencies={"condition": Dependency("decision", "approved")},
)

analysis = TerminateNode(
    name="analysis",
    return_status="analysis",
    output_data=output_data,
    dependencies={"condition": Dependency("decision", "analysis")},
)

denied = TerminateNode(
    name="denied",
    return_status="denied",
    output_data=output_data,
    dependencies={"condition": Dependency("decision", "denied")},
)

policy_with_ds = PolicyDefinition(
    nodes=[
        api,
        decision,
        approved,
        analysis,
        denied,
    ],
    input_schema={"tax_id": "str"},
)

In [ ]:
# Uncomment and run to examine the policy structure
# policy_with_ds.to_dict()

In [ ]:
with Lineart(server_url=BASE_URL) as client:
    version_name = "v1"
    policy_version = client.policy_versions.create(
        policy_id=policy_id, alias=version_name
    )
    policy_version_id = policy_version.policy_version_id

policy_version

In [ ]:
from lineart_sdk.models import WorkflowBase

with Lineart(server_url=BASE_URL) as client:
    policy_version = client.policy_versions.update(
        policy_version_id=policy_version_id,
        alias=version_name,
        workflow=WorkflowBase(
            spec=policy_with_ds.to_dict(),
            requirements=[],
        ),
    )

policy_version

In [ ]:
with Lineart(server_url=BASE_URL) as lineart:
    run = lineart.policy_versions.create_run(
        policy_version_id=policy_version_id, input_data={"tax_id": "1"}
    )

run

Substitute the Data Source with a ConnectionNode

In [ ]:
api = ConnectionNode(
    name=MY_API_DATA_INPUT,
    url="http://testdata:5000",
    method="GET",
    headers={"Content-Type": "application/json"},
    params={"full": True},
    body={"tax_id": "'{{inputs.tax_id}}'"},
    response_type=ResponseType.JSON.value,
    dependencies={"inputs": Dependency(INPUT_NODE)},
)

policy_with_connection = PolicyDefinition(
    nodes=[
        api,
        decision,
        approved,
        analysis,
        denied,
    ],
    input_schema={"tax_id": "str"},
)

In [ ]:
# Uncomment and run to examine the policy structure
# policy_with_connection.to_dict()

In [ ]:
with Lineart(server_url=BASE_URL) as client:
    version_name = "v2"
    new_version = client.policy_versions.create(policy_id=policy_id, alias=version_name)
    new_version_id = new_version.policy_version_id
    new_version = client.policy_versions.update(
        policy_version_id=new_version_id,
        alias=version_name,
        workflow=WorkflowBase(
            spec=policy_with_connection.to_dict(),
            requirements=[],
        ),
    )

new_version

In [ ]:
with Lineart(server_url=BASE_URL) as lineart:
    run = lineart.policy_versions.create_run(
        policy_version_id=new_version_id, input_data={"tax_id": "1"}
    )

run